<h1 style="color:rgb(0,120,170)">344.038, KV Multimedia Search and Retrieval (WS2023/24)</h1>
<h2 style="color:rgb(0,120,170)">Task 3_Group B</h2>

| First Name | Family Name  | Matr.Nr   |
|:-----------|:-------------|:----------|
| Branko     | Paunović     | K12046370 |
| Lukas      | Troyer       | K12006666 |
| Hadi       | Sanaei       | K11733444 |

# Retrieval systems

#### Import datasets, setup helper interfaces

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
np.random.seed(42)
from sklearn.decomposition import PCA

from song import songs
from datasets import datasets, LocalDataset
from retrieval import Retrieval

/home/hd/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


#### Prompt for user input

In [2]:
# Number of similar songs to retrieve
num_top_similar = int(input("Enter the number of top similar tracks: "))

# Example:
#   - Title: Letterman
#   - Artist: Wiz Khalifa
query, query_row_id = songs.prompt_song_query()

Enter the number of top similar tracks:  1000
Enter the song title:  Letterman
Enter the artist name:  Wiz Khalifa


### Early Fusion

In [3]:
df_bert = pd.read_csv('datasets/id_lyrics_bert_mmsr.tsv', sep='\t')
df_musicnn = pd.read_csv('datasets/id_musicnn_mmsr.tsv', sep='\t')
df_mfcc = pd.read_csv('datasets/id_mfcc_bow_mmsr.tsv', sep='\t')

### 1_Early Fusion (Text-Bert & Music-Musicnn) 

In [4]:
# Import the library
from early_fusion import EarlyFusionLibrary

# Create an instance of the library
ef_library = EarlyFusionLibrary()

# Call the early_fusion_and_save (id_early_fusion_mmsr.tsv)
ef_library.early_fusion_and_save(df_bert, df_musicnn)

In [5]:
retN = Retrieval(n=num_top_similar)
Retrieval.create_df_from_tracks(retN.top_similar_tracks(query_row_id, dataset=datasets.early_fusion))

Error syncing cache with disk: Unterminated string starting at: line 1 column 32551791 (char 32551790)


,id,similarity,song,artist,album_name
0,MWGrjYLnFktV2eFf,0.811837,Concept Nothing,Night Lovell,Concept Vague
1,sGD0zOgtbcrwZZO6,0.808541,1997 DIANA,BROCKHAMPTON,1997 DIANA
2,OD6sfVyohxUZAO36,0.807060,Top Back,T.I.,King
3,M65mU1UIozrDxcvu,0.805005,Fast Lane,Bad Meets Evil,Hell: The Sequel (Deluxe)
4,VwApT9MehZ2gx1a8,0.804581,VIVID,BROCKHAMPTON,iridescence
...,...,...,...,...,...
995,xFHmLuGPrHQpfw5b,0.444125,Vai render,Letrux,Letrux em Noite de Climão
996,VxkJOjheEZ2KmXhv,0.444099,Butterfly,Crazy Town,The Gift Of Game
997,k3fIF5XsWbLhdgcu,0.444039,D(r)own,Ghostemane,Hexada
998,1D1TsRmNu7OYSnlG,0.443672,Peachfuzz,KMD,Mr. Hood


### 2_Early Fusion (Text-Bert & Music-Mfcc) 

In [6]:

# Call the early_fusion_and_save (id_early_fusion_mmsr.tsv)
ef_library.early_fusion_and_save(df_bert, df_mfcc)

In [7]:
retN = Retrieval(n=num_top_similar)
Retrieval.create_df_from_tracks(retN.top_similar_tracks(query_row_id, dataset=datasets.mfcc_bow))

Error syncing cache with disk: Unterminated string starting at: line 1 column 32551791 (char 32551790)


,id,similarity,song,artist,album_name
0,QbrBgVgAfI75hR3y,0.938748,Soulja's Story,2Pac,2Pacalypse Now
1,aqGzvg198LcJo0f6,0.938123,Whisper,VIXX LR,2nd Mini Album Whisper
2,ost5WwHhojQYkCoV,0.935841,Best 4 U,Maroon 5,Red Pill Blues (Deluxe)
3,15hcBfdPhrtLVay2,0.934217,DNA,BTS,Love Yourself 承 'Her'
4,iHRt8u88DH1Dq7SP,0.933773,Loyal,Chris Brown,X (Expanded Edition)
...,...,...,...,...,...
995,twZa5028OAb1bOmd,0.879978,Waiting for Superman,Daughtry,Baptized (Deluxe Version)
996,dXJi15Zz5qNzLWmP,0.879964,Less Than I Do,The Band CAMINO,Less Than I Do
997,ivBNExa7WNHOui69,0.879951,Stand By Me,Prince Royce,Prince Royce
998,xHSvLWaBLBPPjTC3,0.879936,Never Let It Go,Afro-Dite,Absolute Svenskt 00-tal


### Late Fusion

In [8]:
dfr_bert = Retrieval.create_df_from_tracks(retN.top_similar_tracks(query_row_id, dataset=datasets.lyrics_bert))
dfr_musicnn= Retrieval.create_df_from_tracks(retN.top_similar_tracks(query_row_id, dataset=datasets.musicnn))
dfr_resnet= Retrieval.create_df_from_tracks(retN.top_similar_tracks(query_row_id, dataset=datasets.resnet))

### 1_Late Fusion - Score Aggregation Fusion for two DataFrames (Text-Bert & Music-Musicnn)

In [9]:

from late_fusion import perform_late_fusion

# Specify fusion method and weights
fusion_method = 'score'  # or 'rank'
d1_weight = 0.6
d2_weight = 0.4

# Perform late fusion
fusion_results_df = perform_late_fusion(dfr_bert, dfr_musicnn, fusion_method, d1_weight, d2_weight)
fusion_results_df


,id,similarity_d1,similarity_d2,song,artist,album_name,aggregated_score
0,5S5J81wX2tJKJnUF,0.736194,0.969205,Mansion,lil skies,Shelby,0.829399
4,OD6sfVyohxUZAO36,0.715089,0.975030,Top Back,T.I.,King,0.819065
3,M65mU1UIozrDxcvu,0.715846,0.959691,Fast Lane,Bad Meets Evil,Hell: The Sequel (Deluxe),0.813384
8,iHRt8u88DH1Dq7SP,0.698288,0.972653,Loyal,Chris Brown,X (Expanded Edition),0.808034
10,VwApT9MehZ2gx1a8,0.694627,0.974256,VIVID,BROCKHAMPTON,iridescence,0.806478
...,...,...,...,...,...,...,...
225,xmczZLFF8u1O5q7H,0.539161,0.836789,They're Red Hot,Red Hot Chili Peppers,Blood Sugar Sex Magik (Deluxe Edition),0.658212
249,nB0Z9UDydK0L8xtl,0.532934,0.843339,Take My Hand,Charli XCX,True Romance,0.657096
250,tmNm9OOiuntHNPhd,0.532881,0.843202,The Limit,Class Actress,Movies,0.657009
251,yRsKR8ectIxgSt78,0.531996,0.843958,Crave you,Flight Facilities,Down To Earth,0.656781


### 2_Late Fusion - Score Aggregation Fusion for two DataFrames (Text-Bert & Video-Resnet)

In [10]:

# Specify fusion method and weights
fusion_method = 'score'  # or 'rank'
d1_weight = 0.6
d2_weight = 0.4

# Perform late fusion
fusion_results_df = perform_late_fusion(dfr_bert, dfr_resnet, fusion_method, d1_weight, d2_weight)
fusion_results_df


,id,similarity_d1,similarity_d2,song,artist,album_name,aggregated_score
1,xTkNXKaWbD5Koq8M,0.692615,0.722439,Money,Leikeli47,Wash & Set,0.704544
10,5i9Rj9XyqhEfSegl,0.666698,0.755106,Barbie Tingz,Nicki Minaj,Barbie Tingz,0.702061
2,jJEgEAF2iuW4yk9g,0.691716,0.712143,When It Rain,Danny Brown,Atrocity Exhibition,0.699887
4,qfdGKqj2QIuogYbc,0.689042,0.701616,Real Ties,lil skies,Real Ties,0.694072
0,NNB2CFD4cHOS0dZp,0.693204,0.689670,Fight Night,Migos,No Label II,0.691790
...,...,...,...,...,...,...,...
128,EOBcqQq4c4URWmPa,0.529619,0.687891,Gun,CHVRCHES,The Bones of What You Believe,0.592928
130,ohbvGoXAJ9BPZk51,0.528986,0.687923,Very Last Moment in Time,Lindsay Lohan,Speak,0.592560
132,Zw4JzSXzOsEjckNB,0.528427,0.684029,Dream On,Amy Macdonald,Under Stars,0.590668
134,CFHZ9aQF6a0zVsOF,0.526439,0.683977,King,Saint PHNX,King,0.589454


### 3_Late Fusion - Score Aggregation Fusion for two DataFrames (Music-Musicnn & Video-Resnet)

In [11]:

# Specify fusion method and weights
fusion_method = 'score'  # or 'rank'
d1_weight = 0.6
d2_weight = 0.4

# Perform late fusion
fusion_results_df = perform_late_fusion(dfr_musicnn, dfr_resnet, fusion_method, d1_weight, d2_weight)
fusion_results_df


,id,similarity_d1,similarity_d2,song,artist,album_name,aggregated_score
20,gw9zZ3xtxoEjmYeB,0.954321,0.777114,Locos,Cypress Hill,Elephants on Acid,0.883438
2,2IZjV6srFH5YXkFr,0.987323,0.727367,Was du Liebe nennst,Bausa,Was Du Liebe nennst,0.883341
6,eVaVqLmvyuWX85Ll,0.980981,0.725396,Glamazon,RuPaul,Glamazon,0.878747
3,qkkV8YBBCwbYkPZ3,0.984984,0.705616,Freelance,Toro y Moi,Outer Peace,0.873237
4,dmj5NdY5r6mVCPjF,0.981746,0.709420,Black and Yellow,Wiz Khalifa,Rolling Papers,0.872816
...,...,...,...,...,...,...,...
167,ATZT90pTOOf3OSTv,0.840292,0.684223,Shake Your Rump,Beastie Boys,Paul's Boutique (20th Anniversary Remastered E...,0.777864
162,Tt2zZfL8oZDkkRJm,0.843371,0.679422,2Shy,Shura,Nothing's Real,0.777791
165,WUQwg3HyiTo7i35A,0.842619,0.679289,Belly of the Beat,Grimes,Art Angels,0.777287
161,JtAe8UfBaRHbIzns,0.843482,0.677979,Black Coffee,All Saints,Saints & Sinners,0.777281


### 4_Late Fusion - Rank-Level Fusion for two DataFrames (Text-Bert & Music-Musicnn)

In [12]:

# Specify fusion method and weights
fusion_method = 'rank'  # or 'score'
d1_weight = 0.6
d2_weight = 0.4

# Perform late fusion
fusion_results_df = perform_late_fusion(dfr_bert, dfr_musicnn, fusion_method, d1_weight, d2_weight)
fusion_results_df


,id,similarity_d1,similarity_d2,song,artist,album_name,aggregated_rank
4,OD6sfVyohxUZAO36,0.715089,0.975030,Top Back,T.I.,King,13.0
18,8P1uWIfvVC9g2Qbs,0.681026,0.987178,"Work Hard, Play Hard",Wiz Khalifa,O.N.I.F.C. (Deluxe),13.8
22,P8qxHDSBDJzT6NwC,0.674538,0.988604,1999 WILDFIRE,BROCKHAMPTON,1999 WILDFIRE,15.0
16,qfdGKqj2QIuogYbc,0.689042,0.980756,Real Ties,lil skies,Real Ties,15.8
0,5S5J81wX2tJKJnUF,0.736194,0.969205,Mansion,lil skies,Shelby,16.2
...,...,...,...,...,...,...,...
263,oYIqRH5nrmdpQTmK,0.528232,0.859447,The Thing I Love,Pixie Lott,Young Foolish Happy (Deluxe Edition),255.6
249,nB0Z9UDydK0L8xtl,0.532934,0.843339,Take My Hand,Charli XCX,True Romance,256.8
251,yRsKR8ectIxgSt78,0.531996,0.843958,Crave you,Flight Facilities,Down To Earth,257.6
250,tmNm9OOiuntHNPhd,0.532881,0.843202,The Limit,Class Actress,Movies,257.8


### 5_Late Fusion - Rank-Level Fusion for two DataFrames (Text-Bert & Video-Resnet)

In [13]:

# Specify fusion method and weights
fusion_method = 'rank'  # or 'score'
d1_weight = 0.6
d2_weight = 0.4

# Perform late fusion
fusion_results_df = perform_late_fusion(dfr_bert, dfr_resnet, fusion_method, d1_weight, d2_weight)
fusion_results_df


,id,similarity_d1,similarity_d2,song,artist,album_name,aggregated_rank
10,5i9Rj9XyqhEfSegl,0.666698,0.755106,Barbie Tingz,Nicki Minaj,Barbie Tingz,7.8
1,xTkNXKaWbD5Koq8M,0.692615,0.722439,Money,Leikeli47,Wash & Set,8.4
2,jJEgEAF2iuW4yk9g,0.691716,0.712143,When It Rain,Danny Brown,Atrocity Exhibition,13.4
9,kFBTJVLf3HZ7gP3A,0.668271,0.715197,Ni**as In Paris,Jay-Z & Kanye West,Top Five (Music From And Inspired By The Motio...,15.2
20,rExcqzEU4MTlkBiK,0.641680,0.736151,Two Words,Kanye West,The College Dropout,16.2
...,...,...,...,...,...,...,...
106,mp0Ex4ZqaSChRm6D,0.537764,0.677629,Don't Tell Me,Blancmange,Mange Tout (Extended Version),112.6
119,Pocetc7AP7RYKjDA,0.533170,0.683102,Shake It Out,Florence + the Machine,Ceremonials (Deluxe Edition),113.6
132,Zw4JzSXzOsEjckNB,0.528427,0.684029,Dream On,Amy Macdonald,Under Stars,119.0
134,CFHZ9aQF6a0zVsOF,0.526439,0.683977,King,Saint PHNX,King,120.6


### 6_Late Fusion - Rank-Level Fusion for two DataFrames (Music-Musicnn & Video-Resnet)

In [14]:

# Specify fusion method and weights
fusion_method = 'rank'  # or 'score'
d1_weight = 0.6
d2_weight = 0.4

# Perform late fusion
fusion_results_df = perform_late_fusion(dfr_musicnn, dfr_resnet, fusion_method, d1_weight, d2_weight)
fusion_results_df


,id,similarity_d1,similarity_d2,song,artist,album_name,aggregated_rank
2,2IZjV6srFH5YXkFr,0.987323,0.727367,Was du Liebe nennst,Bausa,Was Du Liebe nennst,5.8
6,eVaVqLmvyuWX85Ll,0.980981,0.725396,Glamazon,RuPaul,Glamazon,9.4
20,gw9zZ3xtxoEjmYeB,0.954321,0.777114,Locos,Cypress Hill,Elephants on Acid,13.4
4,dmj5NdY5r6mVCPjF,0.981746,0.709420,Black and Yellow,Wiz Khalifa,Rolling Papers,17.4
3,qkkV8YBBCwbYkPZ3,0.984984,0.705616,Freelance,Toro y Moi,Outer Peace,21.2
...,...,...,...,...,...,...,...
159,SgBvRaxOi7XaDEmU,0.849808,0.680685,The Monster,Eminem,The Marshall Mathers LP2,152.0
162,Tt2zZfL8oZDkkRJm,0.843371,0.679422,2Shy,Shura,Nothing's Real,157.0
160,QkGLVkuaVsni5k6x,0.845918,0.678927,Oui Mais... Non,Mylène Farmer,2001-2011,157.8
165,WUQwg3HyiTo7i35A,0.842619,0.679289,Belly of the Beat,Grimes,Art Angels,159.6


# Evaluation

In [15]:
%load_ext autoreload
%autoreload 2

# Obtain necessary information and store for easy interop
from genres import Genres
from utils import get_retrieval_names_for_types

genres = Genres()

# TODO: check if we havent forgotten any constraints
our_chosen_retrievals = [
    # those have to be chosen because of the task description
    "random_baseline",
    "early_fusion",
    "late_fusion",
    "text_tf_idf",
    "text_bert",
    "text_word2vec",
    "musicnn",

    # the ones below have been chosen because they are the best performing ones
    # while still staying within the limits of the task description

    # one of the mfccs
    "mfcc_bow",

    # one of the video features
    "video_resnet",

    # one of the BLFs
    "blf_spectral",

    # one of the i-vectors
    "ivec256"
]


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
  --> No pickled data file found. Performing on-the-fly data computation and saving to 'pickled_state/precomputed_genres.pickle' for later (re-)use.
Found 10094 songs with genre information


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


AttributeError: 'FloatProgress' object has no attribute 'style'

## Accuracy
#### 1. Precision@k & Recall@k

In [ ]:
from precision_recall import PrecisionRecall

pr = PrecisionRecall(genres)
pr.compute()

In [ ]:
# pr.plot_each(our_chosen_retrievals) # i dont think it makes sense to plot
# each one in a separate plot; it just takes up space and is hard to compare

In [ ]:
pr.plot_all_single(our_chosen_retrievals)

#### 2. nDCG@10

In [ ]:
from ndcg import Ndcg

ndcg = Ndcg(genres)
ndcg.compute()
ndcg.plot(our_chosen_retrievals)

## Beyond Accuracy
### 1. Genre Coverage@10

In [ ]:
from genre_coverage import GenreCoverage

genres_coverage = GenreCoverage(genres)
genres_coverage.compute()
genres_coverage.plot(our_chosen_retrievals)

### 2. Genre Diversity@10

In [ ]:
from genre_diversity import GenreDiversity

genres_diversity = GenreDiversity(genres)
genres_diversity.compute()
genres_diversity.plot(our_chosen_retrievals)

## Combined Score

<!-- F_1 score: harmonic mean of precision and recall

We want to achieve a balance between finding similar tracks, but also 

eg that we want diversity high and genre coverage low -->

justification for choosing which values we want to get high or low:
we want to achieve the "opposite" of random baseline - eg 

- precision: random baseline scores low -> we want high precision
- genre diversity: random baseline scores high -> we want low genre diversity

why? we view random baseline as the worst "system", because there is no systematic approach behind it

we want to find a balance between classical and user-centric metrics: we use 2 classical (f1 score + ndcg) and 2 user-centric (genre coverage + genre diversity) metrics; this is why we weight each metric equally (25%)

the report should be divided in 4 category sections + 1 summary section: text, audio, video, fusion; each section starts with a chart composed of combined scores of all available retrieval systems in that category - we give our justification for choosing the systems for the final comparison based on the chart; the summary section contains all other charts with just our chosen features



In [ ]:
from combined_score import CombinedScore

combined_score = CombinedScore(genres)
combined_score.compute()

combined_score.plot(get_retrieval_names_for_types(["random", "audio"]))
combined_score.plot(get_retrieval_names_for_types(["random", "video"]))
combined_score.plot(get_retrieval_names_for_types(["random", "text"]))
combined_score.plot(get_retrieval_names_for_types(["random", "fusion"]))
combined_score.plot(our_chosen_retrievals)

# Frontend

In [ ]:
# precompute retrievals (for frontend and chached lookup)

# only do this if you want to recompute the retrieval

#retrieval = Retrieval(n=100)
#retrieval.precompute_all(threads=4)

In [ ]:
from utils import write_song_df_to_json_file

write_song_df_to_json_file("frontend/static/songMeta.json", datasets.information.df, datasets.url.df, datasets.genres.df)

# frontend uses frontend/static/songMeta.json and retrievals/*.json
